In [27]:
import pygraphviz as pgv
from IPython.display import Image
import pandas as pd
from sklearn.tree import DecisionTreeRegressor # Import Decision Tree Regressor
from sklearn.tree import export_graphviz

In [5]:
# Read in data
fpath = r'../data/DataMatrices/VansjøData_%s_forBN.csv'

# Season list
season_li = ['summ','aut']

df_dict = {}
for season in season_li:
    infile = fpath %season
    df_dict[season] = pd.read_csv(infile, index_col = 0)

In [6]:
df_dict['summ'].head()

,chla,chla_prevSummer,colour,colour_prevSummer,Cyano,Rain_lag1,Rainy_days_winter,TP,TP_prevSummer,WindDays_under_Q0.4,WindDays_over_Q0.6
Year,,,,,,,,,,,
1980,12.169127,NaN,NaN,NaN,NaN,80.992741,NaN,36.084127,NaN,43.0,27.0
1981,7.075000,16.471230,NaN,NaN,NaN,105.457323,102.0,23.583333,43.573016,28.0,43.0
1982,5.466667,8.331250,NaN,NaN,NaN,175.810938,105.0,37.777778,28.833333,25.0,41.0
1983,5.980000,5.847917,19.4,26.666667,NaN,122.882007,121.0,31.960000,28.336806,23.0,51.0
1984,10.230000,6.073333,NaN,17.033333,NaN,74.093285,121.0,27.360000,29.055000,17.0,51.0


In [7]:
df_dict['aut'].head()

,chla,chla_lag1,chla_prevSummer,colour,colour_lag1,colour_prevSummer,Cyano,Cyano_lag1,Rain_lag1,Rainy_days_winter,TP,TP_lag1,TP_prevSummer,WindDays_under_Q0.4,WindDays_over_Q0.6
Year,,,,,,,,,,,,,,,
1980,20.773333,12.169127,NaN,NaN,NaN,NaN,NaN,NaN,242.199218,NaN,51.061905,36.084127,NaN,38.0,33.0
1981,9.587500,7.075000,16.471230,NaN,NaN,NaN,NaN,NaN,282.257779,102.0,34.083333,23.583333,43.573016,41.0,29.0
1982,6.229167,5.466667,8.331250,26.666667,NaN,NaN,NaN,NaN,151.006233,105.0,18.895833,37.777778,28.833333,28.0,47.0
1983,6.166667,5.980000,5.847917,14.666667,19.4,26.666667,NaN,NaN,182.728817,121.0,26.150000,31.960000,28.336806,25.0,51.0
1984,11.950000,10.230000,6.073333,NaN,NaN,17.033333,NaN,NaN,255.820629,121.0,25.695000,27.360000,29.055000,35.0,31.0


In [46]:
df_dict[season].head()

,chla,chla_prevSummer,colour,colour_prevSummer,Cyano,Rain_lag1,Rainy_days_winter,TP,TP_prevSummer,WindDays_under_Q0.4,WindDays_over_Q0.6
Year,,,,,,,,,,,
1980,12.169127,NaN,NaN,NaN,NaN,80.992741,NaN,36.084127,NaN,43.0,27.0
1981,7.075000,16.471230,NaN,NaN,NaN,105.457323,102.0,23.583333,43.573016,28.0,43.0
1982,5.466667,8.331250,NaN,NaN,NaN,175.810938,105.0,37.777778,28.833333,25.0,41.0
1983,5.980000,5.847917,19.4,26.666667,NaN,122.882007,121.0,31.960000,28.336806,23.0,51.0
1984,10.230000,6.073333,NaN,17.033333,NaN,74.093285,121.0,27.360000,29.055000,17.0,51.0


In [47]:
xy_dict = {'summ':
              {'TP': ['TP_prevSummer'],
               'colour': ['colour_prevSummer','Rain_lag1'],
               'chla': ['WindDays_over_Q0.6','TP','chla_prevSummer','Rainy_days_winter'],
               'Cyano': ['chla','colour','WindDays_under_Q0.4']},
           
           'aut':
              {'TP': ['TP_lag1'],
               'colour': ['colour_lag1','Rain_lag1'],
               'chla': ['WindDays_over_Q0.6','TP','chla_lag1','Rainy_days_winter'],
               'Cyano': ['chla','colour','WindDays_under_Q0.4']}
          }  

In [ ]:
for season in season_li:
    for var in xy_dict[season].keys():
        temp_df = df_dict[season][[var]+xy_dict[season][var]].dropna()
        y = temp_df[var] # Target variable
        X = temp_df[xy_dict[season][var]] # Feature(s)
        regressor = DecisionTreeRegressor(random_state=0)
        regressor.fit(X,y)

        # export the decision tree to a tree.dot file for visualizing the plot easily anywhere
        dot_fpath = r'../Figures/Discretization_regTrees/%s_%s.dot' %(season,var)
        export_graphviz(regressor, out_file =dot_fpath, 
                        feature_names = xy_dict[season][var])

        # Convert .dot file to .png
        png_fpath = r'../Figures/Discretization_regTrees/%s_%s.png' %(season,var)
        pgv.AGraph(dot_fpath).draw(png_fpath, format='png', prog='dot')

In [76]:
for season in season_li:
    for targetvar in xy_dict[season].keys():
        for xvar in xy_dict[season][targetvar]:
            temp_df = df_dict[season][[targetvar]+[xvar]].dropna()
            y = temp_df[targetvar].values # Target variable data
            y = y.reshape(-1, 1)
            X = temp_df[xvar].values # Explanatory var data
            X = X.reshape(-1,1)
            regressor = DecisionTreeRegressor(random_state=0)
            regressor.fit(X,y)

            # export the decision tree to a tree.dot file for visualizing the plot easily anywhere
            dot_fpath = r'../Figures/Discretization_regTrees/%s_%s-%s.dot' %(season,targetvar,xvar)
            export_graphviz(regressor, out_file =dot_fpath, 
                            feature_names = [xvar])

            # Convert .dot file to .png
            png_fpath = r'../Figures/Discretization_regTrees/%s_%s-%s.png' %(season,targetvar,xvar)
            pgv.AGraph(dot_fpath).draw(png_fpath, format='png', prog='dot')